In [46]:
import pandas as pd
import sqlite3
import requests
import os
import datetime

In [47]:
# Load the fire data CSV file
file_path = '/Users/seminmahn/Downloads/Historical_Fire_Data.csv'
fire_data = pd.read_csv(file_path)
fire_data.head()

,AddressStandardized,AlarmDateTimeAmericaNY,CauseOfIgnition,FireAreaOfOrigin,FireType,HeatSource,IncidentDurationMinutes,ItemFirstIgnited,Latitude,Longitude,PropertyLossDollars,PropertyUse,UnitFirstArrive,UnitFirstResponseTimeMinutes,Year,ZipCode,OBJECTID,IncidentID
0,1613 MERIDIAN ST,2023/06/28 16:47:39+00,Intentional,Undetermined,"Outside rubbish, trash or waste fire",Match,10.0,"Magazine, newspaper, writing paper",38.01425,-78.48261,NaN,1 or 2 family dwelling,TO10,3.3,2023.0,22902,1,2023-00004918
1,1014 RIDGE ST,2023/06/28 00:55:59+00,Unintentional,"Laundry area, wash house (laundry)",Building fire,Radiated or conducted heat from operating equi...,45.0,"Dust, fiber, lint, including sawdust and excel...",38.01803,-78.49474,7500.0,Multifamily dwelling,E5,3.8,2023.0,22902,2,2023-00004909
2,38 SPRING CT,2023/06/27 10:43:22+00,NaN,NaN,Building fire,NaN,94.0,NaN,38.08284,-78.48864,0.0,Multifamily dwelling,E1,8.8,2023.0,22901,3,2023-00004894
3,1819 RIVER INN LN,2023/06/22 16:24:04+00,NaN,NaN,"Cooking fire, confined to container",NaN,11.0,NaN,38.08854,-78.44688,NaN,1 or 2 family dwelling,E1,9.7,2023.0,22901,4,2023-00004744
4,637 OAK HILL CT,2023/06/17 05:17:59+00,NaN,NaN,Building fire,NaN,91.0,NaN,38.00438,-78.52268,25000.0,1 or 2 family dwelling,E10,7.3,2023.0,22902,5,2023-00004596


In [48]:
def fetch_data_from_url(url, output_file_path):
    try:
        response = requests.get(url)
        response.raise_for_status()  
        with open(output_file_path, 'wb') as file:
            file.write(response.content)
        print(f"Data fetched and saved to {output_file_path}")
    except Exception as e:
        print(f"Error fetching data from URL: {e}")

# Example usage
test_url = "https://opendata.charlottesville.org/datasets/31093f4678b44cc38e0e8347b2336e44_32/explore"
fetched_file_path = 'fetched_fire_data.csv'  

fetch_data_from_url(test_url, fetched_file_path)

Data fetched and saved to fetched_fire_data.csv


In [49]:
# Function to handle errors and report failures in any of the steps
def safe_execution(func):
    try:
        func()
    except Exception as e:
        print(f"Error encountered: {e}")
        return False
    return True

In [50]:
# Step 1: Conversion functions (CSV to JSON, CSV to SQL)
def convert_to_json(dataframe, output_path):
    """Convert dataframe to JSON."""
    dataframe.to_json(output_path, orient='records', lines=True)

def convert_to_sql(dataframe, db_path, table_name):
    """Convert dataframe to SQL table."""
    conn = sqlite3.connect(db_path)
    dataframe.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

In [66]:
# Step 2: Modifying columns (selected key columns)
selected_columns = ['AlarmDateTimeAmericaNY', 'CauseOfIgnition', 'FireType', 'PropertyLossDollars', 'ZipCode']
modified_fire_data = fire_data[selected_columns]

In [68]:
modified_fire_data.loc[:, 'ProcessingDate'] = datetime.datetime.now().strftime("%Y-%m-%d")

/var/folders/tc/94tjsd711558sc5mhtsxz9m40000gn/T/ipykernel_92549/2670080096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_fire_data.loc[:, 'ProcessingDate'] = datetime.datetime.now().strftime("%Y-%m-%d")


In [70]:
output_json_path = '/Users/seminmahn/Downloads/Historical_Fire_Data.json'
output_db_path = '/Users/seminmahn/Downloads/Historical_Fire_Data.db'
table_name = 'fire_data'

In [72]:
# Define output paths and table name
output_json_path = '/Users/seminmahn/Downloads/Historical_Fire_Data.json'
output_db_path = '/Users/seminmahn/Downloads/Historical_Fire_Data.db'
table_name = 'fire_data'

# Store the data in JSON format
safe_execution(lambda: convert_to_json(modified_fire_data, output_json_path))

# Store the data in an SQLite database
safe_execution(lambda: convert_to_sql(modified_fire_data, output_db_path, table_name))


True

In [55]:
# Step 4: Generating summary report before and after transformation
initial_summary = {
    "Number of records": fire_data.shape[0],
    "Number of columns": fire_data.shape[1],
}

modified_summary = {
    "Number of records": modified_fire_data.shape[0],
    "Number of columns": modified_fire_data.shape[1],
}

initial_summary, modified_summary

modified_fire_data.head()

,AlarmDateTimeAmericaNY,CauseOfIgnition,FireType,PropertyLossDollars,ZipCode,ProcessingDate
0,2023/06/28 16:47:39+00,Intentional,"Outside rubbish, trash or waste fire",NaN,22902,2024-10-17
1,2023/06/28 00:55:59+00,Unintentional,Building fire,7500.0,22902,2024-10-17
2,2023/06/27 10:43:22+00,NaN,Building fire,0.0,22901,2024-10-17
3,2023/06/22 16:24:04+00,NaN,"Cooking fire, confined to container",NaN,22901,2024-10-17
4,2023/06/17 05:17:59+00,NaN,Building fire,25000.0,22902,2024-10-17
